# Transformerの要素

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

## ScaledDotProductAttentionクラス


In [1]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.scaling_factor = torch.rsqrt(torch.tensor(d_k, dtype=torch.float))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q (Tensor): Queries tensor, shape [batch_size, n_head, seq_len, d_k].
            K (Tensor): Keys tensor, shape [batch_size, n_head, seq_len, d_k].
            V (Tensor): Values tensor, shape [batch_size, n_head, seq_len, d_v].
            mask (Tensor, optional): Mask tensor, shape [batch_size, 1, 1, seq_len].

        Returns:
            Tensor: Output tensor, shape [batch_size, n_head, seq_len, d_v].
            Tensor: Attention weights tensor, shape [batch_size, n_head, seq_len, seq_len].
        """

        # Compute scaled dot-product attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scaling_factor

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 1, float('-inf'))

        # Compute attention weights
        attn_weights = self.softmax(attn_scores)

        # Compute weighted sum of values
        output = torch.matmul(attn_weights, V)

        return output, attn_weights

### ScaledDotProductAttentionを使ってみる

In [ ]:
# ハイパーパラメータ
batch_size = 16
n_head = 8
seq_len = 10
d_k = 64
d_v = 128

# ScaledDotProductAttentionモジュールのインスタンス化
scaled_dot_product_attention = ScaledDotProductAttention(d_k)

# ランダムなテンソルを生成
Q = torch.randn(batch_size, n_head, seq_len, d_k)
K = torch.randn(batch_size, n_head, seq_len, d_k)
V = torch.randn(batch_size, n_head, seq_len, d_v)
print(f'ランダムQのサイズ: {Q.shape}')
print(f'ランダムKのサイズ: {K.shape}')
print(f'ランダムVのサイズ: {V.shape}')

# マスクの作成
# このマスクは、最初の5つの位置だけをアンマスクし、残りの位置をマスクします。
mask = torch.ones(batch_size, 1, 1, seq_len)
mask[:, :, :, :5] = 0

# forwardメソッドを呼び出し
output, attn_weights = scaled_dot_product_attention(Q, K, V, mask)

# 出力とAttention weightを表示
print(f'出力のサイズ: {output.size()}')  # 出力テンソルのサイズを表示: [batch_size, n_head, seq_len, d_v]
print(f'Attention Weightのサイズ: {attn_weights.size()}')  # Attention weightテンソルのサイズを表示: [batch_size, n_head, seq_len, seq_len]

ランダムQのサイズ: torch.Size([16, 8, 10, 64])
ランダムKのサイズ: torch.Size([16, 8, 10, 64])
ランダムVのサイズ: torch.Size([16, 8, 10, 128])
出力のサイズ: torch.Size([16, 8, 10, 128])
Attention Weightのサイズ: torch.Size([16, 8, 10, 10])


## MultiHeadAttentionクラス

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head, d_k, d_v):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.W_Q = nn.Linear(d_model, d_k * n_head, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_head, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_head, bias=False)
        self.W_O = nn.Linear(d_v * n_head, d_model, bias=False)

        self.attention = ScaledDotProductAttention(d_k)

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        # print(f'W_Q直後のサイズ： {self.W_Q(Q).shape}')
        # print(f'W_K直後のサイズ： {self.W_K(K).shape}')
        # print(f'W_V直後のサイズ： {self.W_V(V).shape}')

        # Linear projections
        Q = self.W_Q(Q).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
        K = self.W_K(K).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
        V = self.W_V(V).view(batch_size, -1, self.n_head, self.d_v).transpose(1, 2)
        # print(f'Qのサイズ： {Q.shape}')
        # print(f'Kのサイズ： {K.shape}')
        # print(f'Vのサイズ： {V.shape}')

        if mask is not None:
            mask = mask.unsqueeze(1)  # [batch_size, 1, seq_len, seq_len]

        # Apply Scaled Dot Product Attention
        x, attn = self.attention(Q, K, V, mask=mask)  # [batch_size, n_head, seq_len, d_v]

        # Concatenate and apply final linear
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.n_head * self.d_v)  # [batch_size, seq_len, n_head * d_v]
        output = self.W_O(x)  # [batch_size, seq_len, d_model]

        return output, attn

### MultiHeadAttentionを使ってみる

In [ ]:
# ハイパーパラメータ
batch_size = 16
seq_len = 10
d_model = 512  # 入力特徴の次元数
n_head = 8  # Attention headの数
d_k = 64  # キー/クエリベクトルの次元数
d_v = 128  # 値ベクトルの次元数

# MultiHeadAttentionモジュールのインスタンス化
multi_head_attention = MultiHeadAttention(d_model, n_head, d_k, d_v)

# ランダムなテンソルを生成
Q = torch.randn(batch_size, seq_len, d_model)
K = torch.randn(batch_size, seq_len, d_model)
V = torch.randn(batch_size, seq_len, d_model)
print(f'ランダムQのサイズ: {Q.shape}')
print(f'ランダムKのサイズ: {K.shape}')
print(f'ランダムVのサイズ: {V.shape}')

# オプショナル: マスクの作成
# このマスクは、最初の5つの位置だけをアンマスクし、残りの位置をマスクします。
mask = torch.ones(batch_size, 1, seq_len)
mask[:, :, :5] = 0

# forwardメソッドを呼び出し
output, attn_weights = multi_head_attention(Q, K, V, mask)

# 出力とAttention weightを表示
print(f'出力のサイズ: {output.size()}')  # 出力テンソルのサイズを表示: [batch_size, seq_len, d_model]
print(f'Attention Weightのサイズ: {attn_weights.size()}')  # Attention weightテンソルのサイズを表示: [batch_size, n_head, seq_len, seq_len]

ランダムQのサイズ: torch.Size([16, 10, 512])
ランダムKのサイズ: torch.Size([16, 10, 512])
ランダムVのサイズ: torch.Size([16, 10, 512])
W_Q直後のサイズ： torch.Size([16, 10, 512])
W_K直後のサイズ： torch.Size([16, 10, 512])
W_V直後のサイズ： torch.Size([16, 10, 1024])
Qのサイズ： torch.Size([16, 8, 10, 64])
Kのサイズ： torch.Size([16, 8, 10, 64])
Vのサイズ： torch.Size([16, 8, 10, 128])
出力のサイズ: torch.Size([16, 10, 512])
Attention Weightのサイズ: torch.Size([16, 8, 10, 10])


## PositionalEncodingクラス
Section3で構築した「Positional Encoding」のクラスです。

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        """
        Args:
            x (Tensor): Input tensor, shape [batch_size, seq_len, d_model]

        Returns:
            Tensor: Output tensor, shape [batch_size, seq_len, d_model]
        """
        # Add positional encoding to the input tensor
        x = x + self.encoding[:, :x.size(1), :]
        return x

### PositionalEncodingを使ってみる

In [ ]:
# ハイパーパラメータの設定
batch_size = 16
seq_len = 10
d_model = 512  # モデルの次元数

# PositionalEncodingモジュールのインスタンス化
positional_encoding = PositionalEncoding(d_model)

# ランダムなテンソルを生成
x = torch.randn(batch_size, seq_len, d_model)

# forwardメソッドを呼び出し
encoded_x = positional_encoding(x)

# エンコードされたデータの表示
print(encoded_x.size())  # エンコードされたテンソルのサイズを表示: [batch_size, seq_len, d_model]

# エンコードされたポジショナルエンコーディングをNumPy配列に変換
print(encoded_x.size())
print(encoded_x.squeeze(0).shape)
encoded_x_0 = encoded_x[0].squeeze(0).detach().numpy()
print(encoded_x_0.shape)
# グラフをプロット
plt.figure(figsize=(10, 10))
plt.pcolormesh(encoded_x_0.transpose(), cmap='viridis')
plt.xlabel('Sequence Position')
plt.ylabel('Embedding Dimension')
plt.colorbar()
plt.show()

torch.Size([16, 10, 512])


## PositionwiseFeedForwardクラス

In [4]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        Args:
            d_model (int): The dimension of the model (also the input and output dimension).
            d_ff (int): The dimension of the feed-forward hidden layer.
            dropout (float): Dropout probability.
        """
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Args:
            x (Tensor): Input tensor, shape [batch_size, seq_len, d_model]

        Returns:
            Tensor: Output tensor, shape [batch_size, seq_len, d_model]
        """
        return self.w_2(self.dropout(self.relu(self.w_1(x))))

### PositionwiseFeedForwardを使ってみる

In [ ]:
# ハイパーパラメータの設定
batch_size = 16
seq_len = 10
d_model = 512  # モデルの次元数
d_ff = 2048  # フィードフォワード隠れ層の次元数

# PositionwiseFeedForwardモジュールのインスタンス化
positionwise_ff = PositionwiseFeedForward(d_model, d_ff)

# ランダムなテンソルを生成
x = torch.randn(batch_size, seq_len, d_model)

# forwardメソッドを呼び出し
output = positionwise_ff(x)

# 出力の表示
print(output.size())  # 出力テンソルのサイズを表示: [batch_size, seq_len, d_model]

torch.Size([16, 10, 512])


## Token Embeddingを使ってみる
Token Embeddingは、PyTorchの`nn.Embedding()`を使いを実装することができます。  
`nn.Embedding()`は通常、単語の埋め込み（単語ベクトル）を学習するために自然言語処理タスクで使用されます。  
埋め込みベクトルのテンソルの形状は[5, 50]となり、5つの単語それぞれに対して50次元の埋め込みベクトルが得られています。  
訓練中にこれらの埋め込みベクトルはバックプロパゲーションを使って更新されます。  
これにより、タスクの性能が次第に向上します。  

In [ ]:
# ハイパーパラメータの設定
vocab_size = 10  # 語彙サイズ
embed_dim = 50   # 埋め込みベクトルの次元数

# nn.Embeddingのインスタンスを作成
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)

# 入力データ（単語IDのテンソル）
input_data = torch.tensor([0, 1, 2, 3, 4], dtype=torch.long)  # 単語IDの例

# 埋め込み層を通じて入力データを変換
embedded_data = embedding_layer(input_data)

# 埋め込まれたデータの表示
print(embedded_data.shape)
print(embedded_data[0])


torch.Size([5, 50])
tensor([ 0.6330, -0.0159, -1.8124,  1.3223, -0.0433, -1.6019,  1.0942, -1.6316,
        -0.4016, -0.0567, -0.0388,  0.7298, -0.5838, -1.6775,  0.3801, -0.5028,
        -1.2791,  0.0763, -1.4816,  0.6320,  0.3060,  0.8212, -0.3067, -0.4664,
        -0.2831, -1.4834,  0.5508,  1.7823,  0.3094,  0.5186,  1.1456, -0.1017,
         1.0952, -0.9636,  0.7506,  0.2427, -0.2452,  0.6169,  1.3137,  0.6934,
        -0.1556,  1.1358, -0.6221, -1.4098,  0.1731, -1.1881,  0.7409,  0.8279,
         1.0083,  0.2723], grad_fn=<SelectBackward0>)


## Layer Normalizationを使ってみる

In [ ]:
# ハイパーパラメータの設定
d_model = 512  # モデルの次元数
eps = 1e-6  # イプシロン（数値安定性のため）

# LayerNormのインスタンス化
layer_norm = nn.LayerNorm(normalized_shape=d_model, eps=eps)

# ランダムなテンソルを生成
batch_size = 16
seq_len = 10
x = torch.randn(batch_size, seq_len, d_model)

# LayerNormを適用
normalized_x = layer_norm(x)

# 結果の表示
print(normalized_x.size())  # 出力テンソルのサイズを表示: [batch_size, seq_len, d_model]

torch.Size([16, 10, 512])


# 要素を使ってTransformerを構築

## EncoderLayerクラス

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_k, d_v, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_head, d_k, d_v)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        # Self-attention sublayer
        attn_output, _ = self.self_attn(x, x, x, mask)
        x = x + self.dropout(attn_output) # 迂回してきたものと足し合わせる
        x = self.layer_norm1(x)

        # Feed-forward sublayer
        ff_output = self.feed_forward(x)
        x = x + self.dropout(ff_output) # 迂回してきたものと足し合わせる
        x = self.layer_norm2(x)

        return x

## Encoderクラス

In [7]:
class Encoder(nn.Module):
    def __init__(self, d_model, n_head, d_k, d_v, d_ff, num_layers, dropout=0.1):
        super(Encoder, self).__init__()
        # 層の数だけ繰り返すためのリストを作成
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_head, d_k, d_v, d_ff, dropout) for _ in range(num_layers)])
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        x = self.layer_norm(x)
        return x

### Encoderを使ってみる

In [8]:
# パラメータ設定
d_model = 512  # 埋め込みの次元数
n_head = 8     # アテンションヘッドの数
d_k = d_v = 64 # キーと値の次元数
d_ff = 2048    # フィードフォワードネットワークの内部次元数
num_layers = 6 # エンコーダの層の数
batch_size = 32  # バッチサイズ
seq_len = 100  # 入力シーケンスの長さ
dropout = 0.1  # ドロップアウト率

# エンコーダのインスタンス化
encoder = Encoder(d_model, n_head, d_k, d_v, d_ff, num_layers, dropout)

# モデルを評価モードに設定（ドロップアウトなどが無効になる）
encoder.eval()

# ダミーの入力データとマスクの生成
input_tensor = torch.rand(batch_size, seq_len, d_model)  # ダミーの入力テンソル
mask = torch.zeros(batch_size, seq_len, seq_len)  # マスク（ここでは全てマスクなし）

# エンコーダに入力データを通す
with torch.no_grad():  # 勾配計算を行わない
    encoded_output = encoder(input_tensor, mask)

print("Encoded output shape:", encoded_output.shape)  # 出力テンソルの形状を表示: [batch_size, seq_len, d_model]
print("Encoded output:", encoded_output[0])

Encoded output shape: torch.Size([32, 100, 512])
Encoded output: tensor([[-0.2766, -0.2822,  0.8418,  ..., -1.8167,  0.0326,  1.2456],
        [ 0.8136, -0.4500, -0.4342,  ...,  0.2924,  0.0264,  0.6845],
        [ 1.7041, -1.0821, -1.0971,  ...,  0.0374,  0.1104,  1.4511],
        ...,
        [ 1.9274,  0.4755, -0.0206,  ..., -2.0815, -0.7704, -0.1266],
        [ 2.0487, -1.3530,  1.4767,  ...,  0.1671, -0.9328,  0.7408],
        [ 0.6506, -1.2212,  0.6088,  ..., -0.3580, -0.9301,  0.1227]])


## DecoderLayerクラス

In [9]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_k, d_v, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_head, d_k, d_v)
        self.enc_dec_attn = MultiHeadAttention(d_model, n_head, d_k, d_v)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.layer_norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        # Self-attention sublayer with target mask
        self_attn_output, _ = self.self_attn(x, x, x, tgt_mask)
        x = x + self.dropout(self_attn_output)
        x = self.layer_norm1(x)

        # Encoder-decoder attention sublayer with source mask
        enc_dec_attn_output, _ = self.enc_dec_attn(x, enc_output, enc_output, src_mask)
        x = x + self.dropout(enc_dec_attn_output)
        x = self.layer_norm2(x)

        # Feed-forward sublayer
        ff_output = self.feed_forward(x)
        x = x + self.dropout(ff_output)
        x = self.layer_norm3(x)

        return x

## Decoderクラス

In [10]:
class Decoder(nn.Module):
    def __init__(self, d_model, n_head, d_k, d_v, d_ff, num_layers, dropout=0.1):
        super(Decoder, self).__init__()
        # 層の数だけ繰り返すためのリストを作成
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_head, d_k, d_v, d_ff, dropout) for _ in range(num_layers)])
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, enc_output, src_mask, tgt_mask)
        x = self.layer_norm(x)
        return x

### Dcoderを使ってみる

In [11]:
# パラメータ設定
d_model = 512  # 埋め込みの次元数
n_head = 8     # アテンションヘッドの数
d_k = d_v = 64 # キーと値の次元数
d_ff = 2048    # フィードフォワードネットワークの内部次元数
num_layers = 6 # デコーダの層の数
batch_size = 32  # バッチサイズ
tgt_seq_len = 100  # デコーダへの入力のシーケンスの長さ
src_seq_len = 120  # エンコーダからの出力のシーケンスの長さ
dropout = 0.1  # ドロップアウト率

# デコーダのインスタンス化
decoder = Decoder(d_model, n_head, d_k, d_v, d_ff, num_layers, dropout)

# モデルを評価モードに設定（ドロップアウトなどが無効になる）
decoder.eval()

# ダミーの入力
input_tensor = torch.rand(batch_size, tgt_seq_len, d_model)  # デコーダへの入力
enc_output = torch.rand(batch_size, src_seq_len, d_model)    # エンコーダからの出力

# マスクの設定
src_mask = torch.zeros(batch_size, tgt_seq_len, src_seq_len)
tgt_mask = torch.triu(torch.ones(batch_size, tgt_seq_len, tgt_seq_len), diagonal=1)  # 一部を0に

# デコーダにデータを通す
with torch.no_grad():  # 勾配計算を行わない
    decoded_output = decoder(input_tensor, enc_output, src_mask, tgt_mask)

print("Decoded output shape:", decoded_output.shape)  # 出力テンソルの形状を表示: [batch_size, tgt_seq_len, d_model]
print("Decoded output:", decoded_output[0])

Decoded output shape: torch.Size([32, 100, 512])
Decoded output: tensor([[ 0.3445,  0.2929,  0.7127,  ..., -0.0193,  1.6462, -1.7966],
        [-1.1351, -0.1350,  1.6325,  ..., -0.1812,  0.9246, -1.1975],
        [-0.5609,  0.8292, -0.7849,  ..., -1.0452,  0.1029, -0.9809],
        ...,
        [-0.3113,  0.7378,  0.5019,  ..., -1.8323,  2.0693,  0.9052],
        [ 0.8796, -0.8885,  1.6949,  ..., -0.4657,  1.9662,  0.6901],
        [ 0.0764,  0.5260, -0.1914,  ...,  0.9847,  1.0884,  1.0874]])


## Transformerクラス

In [12]:
class Transformer(nn.Module):
    def __init__(self, d_model, n_head, d_k, d_v, d_ff, num_encoder_layers, num_decoder_layers,
                 src_vocab_size, tgt_vocab_size, max_src_seq_len, max_tgt_seq_len, dropout=0.1):
        super(Transformer, self).__init__()

        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_src_seq_len)
        self.encoder = Encoder(d_model, n_head, d_k, d_v, d_ff, num_encoder_layers, dropout)

        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_decoder = PositionalEncoding(d_model, max_tgt_seq_len)
        self.decoder = Decoder(d_model, n_head, d_k, d_v, d_ff, num_decoder_layers, dropout)

        self.output_linear = nn.Linear(d_model, tgt_vocab_size)
        self.softmax = nn.Softmax(dim=-1)

        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size
        self.d_model = d_model

    def forward(self, src, tgt, src_mask, tgt_mask):
        src_emb = self.encoder_embedding(src) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float))
        src_emb = self.pos_encoder(src_emb)
        enc_output = self.encoder(src_emb, src_mask)

        tgt_emb = self.decoder_embedding(tgt) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float))
        tgt_emb = self.pos_decoder(tgt_emb)
        dec_output = self.decoder(tgt_emb, enc_output, src_mask, tgt_mask)

        output = self.output_linear(dec_output)
        return self.softmax(output)

## Transformerの訓練をやってみる
簡単な翻訳データセットを作成し、そのデータセットを使用してモデルを訓練します。  
使用する翻訳データセットは非常に小さく、実際の翻訳タスクを学習するには不十分ですが、デモンストレーションのためには十分です。


In [13]:
# 簡単な翻訳データセットを作成する
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab):
        self.src_sentences = src_sentences
        self.tgt_sentences = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        # 文章の長さを取得
        return len(self.src_sentences)

    def __getitem__(self, idx):
        # データローダーでデータを取り出す
        # インデックスに対応した要素
        src_sentence = self.src_sentences[idx]
        tgt_sentence = self.tgt_sentences[idx]
        # それぞれの単語に対応したID
        src_indices = [self.src_vocab[token] for token in src_sentence.split()]
        tgt_indices = [self.tgt_vocab[token] for token in tgt_sentence.split()]
        return torch.tensor(src_indices), torch.tensor(tgt_indices)

# 簡単な語彙とデータセットを作成
src_vocab = {'hello': 0, 'world': 1, 'EOS': 2}
tgt_vocab = {'こんにちは': 0, '世界': 1, 'EOS': 2}
src_sentences = ['hello world EOS', 'world hello EOS']
tgt_sentences = ['こんにちは 世界 EOS', '世界 こんにちは EOS']

# データセットとデータローダーの作成
dataset = TranslationDataset(src_sentences, tgt_sentences, src_vocab, tgt_vocab)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# モデルのインスタンス化
transformer = Transformer(
    d_model=512, n_head=8, d_k=64, d_v=64, d_ff=2048,
    num_encoder_layers=3, num_decoder_layers=3,
    src_vocab_size=len(src_vocab), tgt_vocab_size=len(tgt_vocab),
    max_src_seq_len=10, max_tgt_seq_len=10, dropout=0.1
)

# 損失関数と最適化手法
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer.parameters(), lr=0.001)

# 訓練ループ
for epoch in range(10):  # 実際にはもっと多くのエポックが必要です
    transformer.train()
    total_loss = 0
    for src, tgt in dataloader:
        # モデルの出力を計算
        output = transformer(src, tgt[:, :-1], None, None)

        # 損失を計算
        loss = criterion(output.view(-1, output.size(-1)), tgt[:, 1:].reshape(-1))

        # 勾配を初期化
        optimizer.zero_grad()

        # バックプロパゲーション
        loss.backward()

        # パラメータの更新
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

Epoch 1, Loss: 1.0434155464172363
Epoch 2, Loss: 1.0512516498565674
Epoch 3, Loss: 1.0508849620819092
Epoch 4, Loss: 1.0244863033294678
Epoch 5, Loss: 1.2967438697814941
Epoch 6, Loss: 1.2894434928894043
Epoch 7, Loss: 1.0014259815216064
Epoch 8, Loss: 1.051102876663208
Epoch 9, Loss: 1.05092191696167
Epoch 10, Loss: 1.0528696775436401
